# Async Stream-validate LLM responses


In [5]:
# Few imports and global variables
from rich import print
import guardrails as gd
import litellm
from IPython.display import clear_output
import time

### Setup

Install the necessary validators from Guardrails hub in your CLI.

```bash
!guardrails hub install hub://guardrails/competitor_check
```

#### Create the Guard object


In [6]:
from guardrails.hub import CompetitorCheck
prompt = "Tell me about the Apple Iphone"

guard = gd.AsyncGuard.use(CompetitorCheck, ["Apple"])

ImportError: cannot import name 'CompetitorCheck' from 'guardrails.hub' (/Users/wyatt/Projects/guardrails/guardrails/hub/__init__.py)

##### Example 1: No async streaming

By default, the `stream` parameter is set to `False`. 
We will use LiteLLM to make our LLM calls.


In [ ]:
# Wrap the litellm OpenAI API call with the `guard` object
raw_llm_output, validated_output, *rest = await guard(
    litellm.acompletion,
    model="gpt-3.5-turbo",
    prompt=prompt,
    max_tokens=1024,
    temperature=0.3,
)

In [ ]:
# Let's see the logs
print(guard.history.last.tree)

##### Example 2: Async Streaming

Set the `stream` parameter to `True`


In [ ]:
# Wrap the litellm OpenAI API call with the `guard` object
fragment_generator = await guard(
    litellm.acompletion,
    model="gpt-3.5-turbo",
    prompt=prompt,
    max_tokens=1024,
    temperature=0,
    stream=True,
)


async for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.5)

In [ ]:
# Let's see the logs
print(guard.history.last.tree)

As you can see here, the outputs in both examples match. The only difference is that, in the async streaming example, the outputs are returned as soon as they are received and validated by Guardrails.

 In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. 
 
 In other words, when async streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.
